In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

In [3]:
wine_data = pd.read_csv('data/wine_data.csv')
wine_data.sample(5)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
73,1,12.99,1.67,2.60,30.0,139,3.30,2.89,0.21,1.96,3.35,1.31,3.50,985
69,1,12.21,1.19,1.75,16.8,151,1.85,1.28,0.14,2.50,2.85,1.28,3.07,718
154,2,12.58,1.29,2.10,20.0,103,1.48,0.58,0.53,1.40,7.60,0.58,1.55,640
172,2,14.16,2.51,2.48,20.0,91,1.68,0.70,0.44,1.24,9.70,0.62,1.71,660
65,1,12.37,1.21,2.56,18.1,98,2.42,2.65,0.37,2.08,4.60,1.19,2.30,678


#### Collecting Features

In [4]:
wine_features = wine_data.drop('Class', axis = 1)
wine_target = wine_data[['Class']]

In [5]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(wine_features,
                                                    wine_target,
                                                    test_size=0.4,
                                                    random_state=0)

In [6]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(x_test.values).float()

In [7]:
Ytrain_ = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest_ = torch.from_numpy(y_test.values).view(1,-1)[0]

## Creating a classifier


In [8]:
input_size = 13
output_size = 3
hidden_size = 100

In [9]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [10]:
model = Net()

In [11]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.NLLLoss()

#### Training the model

In [12]:
epochs = 1000

for epoch in range(epochs):

    optimizer.zero_grad()
    Ypred = model(Xtrain_)

    loss = loss_fn(Ypred , Ytrain_)
    loss.backward()

    optimizer.step()
        
    if epoch % 100 == 0:
        print ('Epoch', epoch, 'loss', loss.item())

Epoch 0 loss 1.2193819284439087
Epoch 100 loss 0.20387880504131317
Epoch 200 loss 0.05776870995759964
Epoch 300 loss 0.04551679641008377
Epoch 400 loss 0.038149818778038025
Epoch 500 loss 0.030085008591413498
Epoch 600 loss 0.9084786176681519
Epoch 700 loss 0.5561537146568298
Epoch 800 loss 0.45347943902015686
Epoch 900 loss 0.1811991035938263


## Persist a trained PyTorch model state_dict
* saves and loads only the model parameters
* for training the model later; If you need to keep training the model that you are about to save, you need to save more than just the model. You also need to save the state of the optimizer, epochs, score, etc. 

In [13]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.1154, -0.1269,  0.1410,  ..., -0.0144,  0.0614,  0.2308],
                      [-0.2195, -0.0942,  0.0807,  ..., -0.2271,  0.1546, -0.2261],
                      [ 0.2764,  0.1853,  0.1983,  ...,  0.1809,  0.0441,  0.0069],
                      ...,
                      [ 0.0293,  0.1374, -0.1756,  ...,  0.1414,  0.0112,  0.1909],
                      [ 0.2559,  0.2493,  0.1019,  ..., -0.1858,  0.1827, -0.0974],
                      [-0.2546, -0.1706, -0.0133,  ..., -0.1443,  0.0554, -0.1118]])),
             ('fc1.bias',
              tensor([-0.2213,  0.1422,  0.1815,  0.3115, -0.5826, -0.1345,  0.0807,
                       0.1011,  0.2353,  0.1707,  0.0826,  0.1469, -0.2231, -0.2474,
                       0.3045, -0.1207,  0.1884,  0.1701, -0.1828,  0.0578,  0.0424,
                      -0.1641,  0.0555,  0.2346, -0.1652, -0.1632, -0.0518, -0.0968,
                       0.2446,  0.2596, -0.0284,  0.1617,  0.2419,  0.18

In [14]:
optimizer.state_dict()

{'param_groups': [{'amsgrad': False,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'lr': 0.01,
   'params': [111988212936,
    111988212072,
    90500497480,
    111988126920,
    4570652816,
    111988211928],
   'weight_decay': 0}],
 'state': {4570652816: {'exp_avg': tensor([[ 1.1005e-04,  2.3910e-04,  1.3471e-07,  4.2952e-04,  4.3811e-04,
            -8.6315e-05,  4.5358e-04, -2.7189e-07, -5.9119e-04, -3.6967e-08,
            -5.5829e-04, -6.0118e-04, -2.2506e-04,  3.6496e-04,  4.7876e-04,
             3.5155e-04, -7.3612e-05,  2.3652e-04, -4.2077e-04,  2.3560e-04,
            -1.4567e-05, -4.4893e-08, -7.8507e-07,  1.7912e-04, -3.0631e-04,
            -6.2518e-04,  1.4993e-05,  1.5356e-05, -3.1142e-05,  2.1661e-05,
             7.4309e-10,  3.8105e-04, -8.3421e-05,  7.9729e-06, -4.0890e-04,
            -5.4704e-05,  4.5167e-06, -4.5596e-05,  4.2241e-04, -5.2690e-04,
            -5.9841e-05,  2.4277e-04,  3.8197e-04,  4.1271e-04,  4.1267e-04,
             6.1540e-06, -5.6419e-04,  3

In [16]:
torch.save(model.state_dict(), "models/classifier_state_dict_0.4")

### Loading from state dict

In [15]:
new_model = Net()

In [16]:
new_model.load_state_dict(torch.load('models/classifier_state_dict_'))

In [17]:
new_model.eval()

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

In [18]:
predict_out = new_model(Xtest_)
_, predict_y = torch.max(predict_out, 1)

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print ('prediction accuracy', accuracy_score(Ytest_.data, predict_y.data))
print ('micro precision', precision_score(Ytest_.data, predict_y.data, average='micro'))
print ('micro recall', recall_score(Ytest_.data, predict_y.data, average='micro'))

prediction accuracy 0.9166666666666666
micro precision 0.9166666666666666
micro recall 0.9166666666666666
